In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])  

sk


In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,        
    chunk_overlap=300,      
    separators=[
        "\n【사건개요】",     
        "\n【쟁점사항】",     
        "\n【처리경위】",    
        "\n【처리결과】",   
        "\n■", "\n\n", "\n", ".", " ", ""
    ]
)

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large", 
    dimensions=1536
)


pdf_filepath = 'data/콘텐츠분쟁해결_사례.pdf'
if not os.path.exists(pdf_filepath):
        raise FileNotFoundError(f"파일을 찾을 수 없습니다: {pdf_filepath}")
    
try:
    loader = PyPDFLoader(pdf_filepath)
    documents = loader.load()
    print(f"문서 로드 완료: {len(documents)}개 페이지")
    
    split_documents = text_splitter.split_documents(documents)
    print(f"문서 분할 완료: {len(split_documents)}개 청크")
    
except Exception as e:
    print(f"문서 처리 중 오류 발생: {e}")

embedding_model = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(documents, embeddings)

retriever = vectorstore.as_retriever(
    search_type="similarity",  
    search_kwargs={"k": 5}         
)

llm = ChatOpenAI(
    model="gpt-4o",              
    temperature=0.2,             
    max_tokens=2000              
)


template = """
    당신은 콘텐츠 분야 전문 법률 자문사입니다. 
    아래 분쟁조정 사례들을 바탕으로 정확하고 전문적인 법률 조언을 제공해주세요.

    관련 분쟁사례:
    {context}

    상담 내용: {question}

    답변 가이드라인:
    1. 제시된 사례들을 근거로 답변하세요
    2. 관련 법령이나 조항이 있다면 명시하세요
    3. 비슷한 사례의 처리경위와 결과를 참고하여 설명하세요
    4. 실무적 해결방안을 단계별로 제시하세요
    5. 사례에 없는 내용은 "제시된 사례집에서는 확인할 수 없습니다"라고 명시하세요

    전문 법률 조언:"""
    
prompt = PromptTemplate(
    template=template,
    input_variables=["context", "question"]
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,                        
    chain_type="stuff",             
    retriever=retriever,           
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True   
)


test_questions = [
    "온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?",
    "인터넷 강의를 중도 해지하려고 하는데 과도한 위약금을 요구받고 있습니다. 정당한가요?",
    "무료체험 후 자동으로 유료전환되어 요금이 청구되었습니다. 환불 가능한가요?",
    "미성년자가 부모 동의 없이 게임 아이템을 구매했습니다. 환불받을 수 있는 방법이 있나요?",
    "온라인 교육 서비스가 광고와 다르게 제공되어 계약을 해지하고 싶습니다. 가능한가요?"
]


def classify_dispute_type(query):
    game_keywords = ["게임", "아이템", "계정", "캐릭터", "레벨", "길드", "온라인게임"]
    elearning_keywords = ["강의", "온라인교육", "이러닝", "수강", "환불", "화상교육"]
    web_keywords = ["웹사이트", "무료체험", "자동결제", "구독", "사이트"]
    
    query_lower = query.lower()
    
    if any(keyword in query_lower for keyword in game_keywords):
        return "게임"
    elif any(keyword in query_lower for keyword in elearning_keywords):
        return "이러닝"
    elif any(keyword in query_lower for keyword in web_keywords):
        return "웹콘텐츠"
    else:
        return "기타"
    
for i, question in enumerate(test_questions, 1):
    print(f"\n{'='*60}")
    print(f"질문 {i}: {question}")
    print(f"분쟁 유형: {classify_dispute_type(question)}")
    print(f"{'='*60}")
    
    try:
        result = qa_chain.invoke({"query": question})
        answer = result["result"]
        source_docs = result["source_documents"]
        
        print(f"답변:\n{answer}")
        print(f"\n참조 문서 수: {len(source_docs)}개")
        
        # 참조 문서의 일부 내용 표시
        for j, doc in enumerate(source_docs[:2], 1):  # 상위 2개만 표시
            print(f"\n참조 문서 {j}: {doc.page_content[:200]}...")
            
    except Exception as e:
        print(f"오류 발생: {e}")

문서 로드 완료: 109개 페이지
문서 분할 완료: 104개 청크

질문 1: 온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?
분쟁 유형: 게임
답변:
온라인 게임에서 시스템 오류로 인해 아이템이 사라진 경우, 게임회사가 복구를 거부하는 상황에 대한 법률 조언을 드리겠습니다. 제시된 사례들을 근거로 하여 다음과 같이 답변드립니다.

1. **사례 분석**:
   - **2006년 사례**: 시스템 오류로 아이템이 소멸된 경우, 계정 명의자가 아니면 복구가 불가하다는 게임사의 약관에 따라 복구가 거부되었습니다.
   - **2009년 사례**: 시스템 오류로 인한 게임머니 소실에 대해, 다른 사용자들에게 유사한 불만이 접수되지 않았고, 사용 내역에 문제가 없었다는 이유로 시스템 오류로 보기 어렵다는 결론이 내려졌습니다.
   - **2008년 사례**: 보안 서비스 미작동으로 인한 아이템 도난의 경우, 게임사가 도용을 인정하고 아이템을 복구해 주었습니다.

2. **관련 법령 및 조항**:
   - 민법 제250조 및 제251조는 도품이나 유실물에 대한 반환 청구에 대한 규정을 두고 있으나, 이는 주로 물리적 재산에 대한 것이며, 디지털 아이템에 직접 적용되지는 않습니다.

3. **비슷한 사례의 처리경위 및 결과**:
   - 시스템 오류로 인한 피해의 경우, 게임사의 약관에 따라 계정 명의자 여부가 중요하며, 시스템 오류가 명확히 입증되지 않으면 복구가 어려울 수 있습니다.
   - 보안 서비스의 미작동이 명확히 입증된 경우에는 복구가 이루어질 수 있습니다.

4. **실무적 해결방안**:
   - **1단계: 증거 수집**: 시스템 오류로 인해 아이템이 소실된 증거를 최대한 수집합니다. 게임 내 로그, 스크린샷, 오류 메시지 등을 확보합니다.
   - **2단계: 게임사와의 협의**: 게임사의 고객센터에 정식으로 이의를 제기하고, 수집한 증거를 바탕으로 복구를 요청합니다.
   - **3단